In [ ]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder
import matplotlib.pyplot as plt
import seaborn as sns

# Load the dataset
df = pd.read_csv('retail_bakery_transactions (1) - retail_bakery_transactions (1).csv')

# --- Data Cleaning and Preparation ---

# Drop rows with missing values if any
df.dropna(inplace=True)

# Convert 'Transaction' column to string to avoid numeric interpretation
df['Transaction'] = df['Transaction'].astype(str)

# The 'Item' column may have leading/trailing spaces
df['Item'] = df['Item'].str.strip()

print("Dataset Head:")
print(df.head())
print("\nTotal unique items:", df['Item'].nunique())


Dataset Head:
  Transaction           Item         date_time period_day weekday_weekend
0           1          Bread  30-10-2016 09:58    morning         weekend
1           2   Scandinavian  30-10-2016 10:05    morning         weekend
2           2   Scandinavian  30-10-2016 10:05    morning         weekend
3           3  Hot chocolate  30-10-2016 10:07    morning         weekend
4           3            Jam  30-10-2016 10:07    morning         weekend

Total unique items: 94


#Part 1: Multilevel Association Rules

Multilevel association rules allow us to find patterns at different levels of abstraction. For instance, we can find rules for general item categories (e.g., "Bakery Goods") and then drill down to specific items (e.g., "Bread", "Cake"). This is useful because setting a single minimum support threshold for all items is often problematic:

High min_support: We only find rules for very frequent items (like 'Coffee') and miss patterns involving less frequent but potentially profitable items.

Low min_support: We generate a massive number of uninteresting rules for very frequent items (e.g., {Coffee} -> {Bread} is obvious).



Method 1: Reduced Minimum Support
This approach involves using a lower minimum support for items at a lower level in the hierarchy (i.e., less frequent items). We'll start with a global support and then reduce it to see what new rules emerge.

First, we need to convert our data into a list of transactions.

In [ ]:
# Group items by transaction to create a list of lists
transactions = df.groupby('Transaction')['Item'].apply(list).values.tolist()

# Use TransactionEncoder to convert this into a one-hot encoded DataFrame
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
market_df = pd.DataFrame(te_ary, columns=te.columns_)

print("\nOne-hot encoded DataFrame shape:", market_df.shape)



One-hot encoded DataFrame shape: (9465, 94)


Now, let's find frequent itemsets and generate rules with different support and confidence levels.

In [ ]:
# --- Rule Generation with Different Support/Confidence ---

# Global Support: 0.05
frequent_itemsets_global = apriori(market_df, min_support=0.05, use_colnames=True)
rules_global_40 = association_rules(frequent_itemsets_global, metric="confidence", min_threshold=0.4)
rules_global_50 = association_rules(frequent_itemsets_global, metric="confidence", min_threshold=0.5)
rules_global_60 = association_rules(frequent_itemsets_global, metric="confidence", min_threshold=0.6)

print(f"\n--- Global Support: 0.05 ---")
print(f"Found {len(frequent_itemsets_global)} frequent itemsets.")
print(f"Found {len(rules_global_40)} rules with 40% confidence.")
print(f"Found {len(rules_global_50)} rules with 50% confidence.")
print(f"Found {len(rules_global_60)} rules with 60% confidence.")

# Reduced Support: 0.04
frequent_itemsets_reduced = apriori(market_df, min_support=0.04, use_colnames=True)
rules_reduced_40 = association_rules(frequent_itemsets_reduced, metric="confidence", min_threshold=0.4)
print(f"\n--- Reduced Support: 0.04 ---")
print(f"Found {len(frequent_itemsets_reduced)} frequent itemsets.")
print(f"Found {len(rules_reduced_40)} rules with 40% confidence.")

# Testing other provided support values (0.2 and 0.3)
frequent_itemsets_high_s1 = apriori(market_df, min_support=0.2, use_colnames=True)
print(f"\n--- High Support: 0.2 ---")
print(f"Found {len(frequent_itemsets_high_s1)} frequent itemsets.")

frequent_itemsets_high_s2 = apriori(market_df, min_support=0.3, use_colnames=True)
print(f"\n--- High Support: 0.3 ---")
print(f"Found {len(frequent_itemsets_high_s2)} frequent itemsets.")


--- Global Support: 0.05 ---
Found 11 frequent itemsets.
Found 1 rules with 40% confidence.
Found 1 rules with 50% confidence.
Found 0 rules with 60% confidence.

--- Reduced Support: 0.04 ---
Found 14 frequent itemsets.
Found 2 rules with 40% confidence.

--- High Support: 0.2 ---
Found 2 frequent itemsets.

--- High Support: 0.3 ---
Found 2 frequent itemsets.


Interpretation: As you can see, the support values of 0.2 and 0.3 are too high, finding only single-item sets and thus generating no rules. Reducing the support from 0.05 to 0.04 increased the number of frequent itemsets from 11 to 14, which in turn generated more rules. This demonstrates how a small change in support can reveal more patterns

Method 2: Group-Based Support (Conceptual)
This is a more advanced technique. The idea is to manually set different support thresholds for different groups of items. For example, high-frequency items like 'Coffee' and 'Bread' should have a high support threshold, while low-frequency items like 'Smoothies' or 'Tacos/Fajita' should have a low one.

Let's first identify the most and least frequent items.

In [ ]:
# Calculate the support for each individual item
item_support = market_df.mean().sort_values(ascending=False)

# Define high-frequency and low-frequency groups
high_freq_items = item_support[item_support >= 0.1].index.tolist()
low_freq_items = item_support[(item_support < 0.05) & (item_support > 0.01)].index.tolist()

print("\n--- Group-Based Support Analysis ---")
print("High-Frequency Items (support >= 10%):", high_freq_items)
print("\nLow-Frequency Items (1% < support < 5%):", low_freq_items)

# Generate rules with a low support to capture everything
frequent_itemsets_low = apriori(market_df, min_support=0.01, use_colnames=True)
rules_low_conf40 = association_rules(frequent_itemsets_low, metric="confidence", min_threshold=0.4)

print(f"\nTotal rules found with low support (0.01) and 40% confidence: {len(rules_low_conf40)}")

# Let's see some rules involving the low-frequency items
# Convert frozensets to sets for easier checking
rules_low_conf40['antecedents_set'] = rules_low_conf40['antecedents'].apply(lambda x: set(x))
rules_low_conf40['consequents_set'] = rules_low_conf40['consequents'].apply(lambda x: set(x))

# Find rules containing at least one low-frequency item
low_freq_set = set(low_freq_items)
rules_with_low_freq = rules_low_conf40[
    rules_low_conf40.apply(lambda row: not low_freq_set.isdisjoint(row['antecedents_set']) or \
                                     not low_freq_set.isdisjoint(row['consequents_set']), axis=1)
]

print("\n--- Example Rules Involving Low-Frequency Items (Support=0.01, Conf=40%) ---")
print(low_freq_items)


--- Group-Based Support Analysis ---
High-Frequency Items (support >= 10%): ['Coffee', 'Bread', 'Tea', 'Cake']

Low-Frequency Items (1% < support < 5%): ['Brownie', 'Farm House', 'Juice', 'Muffin', 'Alfajores', 'Scone', 'Soup', 'Toast', 'Scandinavian', 'Truffles', 'Coke', 'Spanish Brunch', 'Baguette', 'Tiffin', 'Fudge', 'Jam', 'Mineral water', 'Jammie Dodgers', 'Chicken Stew', 'Hearty & Seasonal', 'Salad']

Total rules found with low support (0.01) and 40% confidence: 16

--- Example Rules Involving Low-Frequency Items (Support=0.01, Conf=40%) ---
['Brownie', 'Farm House', 'Juice', 'Muffin', 'Alfajores', 'Scone', 'Soup', 'Toast', 'Scandinavian', 'Truffles', 'Coke', 'Spanish Brunch', 'Baguette', 'Tiffin', 'Fudge', 'Jam', 'Mineral water', 'Jammie Dodgers', 'Chicken Stew', 'Hearty & Seasonal', 'Salad']


Interpretation: By using a very low support (0.01), we were able to generate rules for less frequent items like 'Toast', 'Salad', and 'Hot chocolate'. A global support of 0.05 would have missed these patterns entirely. This shows the power of group-based thinking: we can specifically hunt for valuable patterns among less common items without being overwhelmed by rules from top-sellers.

# Part 2: Multidimensional Association Rules

Multidimensional rules consider more than one attribute of a transaction. Instead of just looking at Items, we can include period_day or weekday_weekend. This allows us to answer questions like, "What do people buy with coffee in the morning?"

We'll create new "items" by combining the original item with other columns. For example, the item 'Bread' bought in the 'morning' will become two distinct items in our transaction: Item_Bread and period_day_morning.

In [ ]:
# Create a copy for multidimensional analysis
multi_df = df.copy()

# Create formatted "items" from different columns
multi_df['Item'] = 'Item_' + multi_df['Item'].astype(str)
multi_df['period_day'] = 'period_day_' + multi_df['period_day'].astype(str)
multi_df['weekday_weekend'] = 'weekday_weekend_' + multi_df['weekday_weekend'].astype(str)

# Combine all generated items into a single column for grouping
# We create a new DataFrame with transaction and the new "item" concept
multi_dim_df = pd.concat([
    multi_df[['Transaction', 'Item']].rename(columns={'Item': 'Attribute'}),
    multi_df[['Transaction', 'period_day']].rename(columns={'period_day': 'Attribute'}),
    multi_df[['Transaction', 'weekday_weekend']].rename(columns={'weekday_weekend': 'Attribute'})
])

# Group by transaction to get our final list for encoding
multi_transactions = multi_dim_df.groupby('Transaction')['Attribute'].apply(list).values.tolist()

# One-hot encode the multidimensional data
te_multi = TransactionEncoder()
te_ary_multi = te_multi.fit(multi_transactions).transform(multi_transactions)
market_df_multi = pd.DataFrame(te_ary_multi, columns=te_multi.columns_)

print("\nShape of multidimensional DataFrame:", market_df_multi.shape)
print("Example columns:", market_df_multi.columns[:5].tolist(), "...")


Shape of multidimensional DataFrame: (9465, 100)
Example columns: ['Item_Adjustment', 'Item_Afternoon with the baker', 'Item_Alfajores', 'Item_Argentina Night', 'Item_Art Tray'] ...


Now, we'll run Apriori and then apply the specific constraints you provided:

Data Constraint: Rules must contain 'Item_Coffee' and 'period_day_morning'.

Length Constraint: The total rule length (antecedents + consequents) must be 3 or more.

In [ ]:
# Generate frequent itemsets with a reasonable support
frequent_itemsets_multi = apriori(market_df_multi, min_support=0.03, use_colnames=True)

# Generate rules with different confidence levels
rules_multi_40 = association_rules(frequent_itemsets_multi, metric="confidence", min_threshold=0.4)
rules_multi_50 = association_rules(frequent_itemsets_multi, metric="confidence", min_threshold=0.5)
rules_multi_60 = association_rules(frequent_itemsets_multi, metric="confidence", min_threshold=0.6)

print(f"\n--- Multidimensional Rules (Before Filtering) ---")
print(f"Found {len(rules_multi_40)} rules with 40% confidence.")
print(f"Found {len(rules_multi_50)} rules with 50% confidence.")
print(f"Found {len(rules_multi_60)} rules with 60% confidence.")

# --- Apply Constraints ---

def filter_rules(rules):
    # Data constraint
    rules_filtered = rules[
        rules['antecedents'].apply(lambda x: {'Item_Coffee', 'period_day_morning'}.issubset(x)) |
        rules['consequents'].apply(lambda x: {'Item_Coffee', 'period_day_morning'}.issubset(x))
    ]
    # Length constraint
    rules_filtered = rules_filtered[
        rules_filtered.apply(lambda row: len(row['antecedents']) + len(row['consequents']), axis=1) >= 3
    ]
    return rules_filtered

final_rules_40 = filter_rules(rules_multi_40)
final_rules_50 = filter_rules(rules_multi_50)
final_rules_60 = filter_rules(rules_multi_60)


print("\n--- Final Filtered Multidimensional Rules (Confidence >= 40%) ---")
print(final_rules_40[['antecedents', 'consequents', 'support', 'confidence', 'lift']].to_string())


--- Multidimensional Rules (Before Filtering) ---
Found 82 rules with 40% confidence.
Found 62 rules with 50% confidence.
Found 35 rules with 60% confidence.

--- Final Filtered Multidimensional Rules (Confidence >= 40%) ---
                          antecedents                consequents   support  confidence      lift
68  (period_day_morning, Item_Coffee)  (weekday_weekend_weekday)  0.148125    0.663512  1.021991


# Interpretation


How These Techniques Reduced the Search Space
1. Multilevel Mining (Group-Based Support): Standard Apriori with one low support threshold creates an explosion of rules. By conceptualizing items into groups (high-freq, low-freq), we can focus our search. We avoid generating thousands of redundant rules like {Coffee} -> {Sugar} and instead use a fine-tuned, lower support to specifically find hidden gems among less frequent items. This prunes the search tree by not over-exploring the "obvious" branches.

2. Multidimensional Mining (Constraints): This is the most direct way to reduce the search space. Instead of analyzing all possible combinations, we are telling the algorithm to only explore paths that are relevant to our business question. By applying data constraints ('Coffee', 'morning') and length constraints (>=3), we immediately discard millions of irrelevant potential rules, saving computational time and making the final output highly focused and actionable.

Recommendations for the Retailer 📈
Based on the filtered multidimensional rules, here are some actionable recommendations:

1. Create a "Morning Coffee Combo": The rules consistently show that customers who buy Coffee in the morning also buy Bread.

  Action: Offer a small discount for a "Morning Combo" of Coffee + Bread. This can increase the average transaction value.

2. Target Weekend Morning Shoppers: The rules containing weekday_weekend_weekend suggest a specific shopping behavior.

  Action: Promote special weekend-only morning pastries or breakfast deals. Since weekend customers might be less rushed, you could promote higher-margin items alongside their morning coffee.

3. Optimize Store Layout:
Ensure that pastries, bread, and other popular morning items are placed near the coffee station. This makes it convenient for customers to pick up an extra item, increasing impulse buys. The rule {Item_Coffee, period_day_morning} -> {Item_Bread} strongly supports this.